In [5]:
import getopt
import logging
import os
import sys
# import traceback
import tweepy
import pandas as pd 
import numpy as np
import re
import skimage.io as io  

In [38]:
# Generate your own at https://apps.twitter.com/app
CONSUMER_KEY = 'q8svcQ1GKW2yknY8MCZLvcO7w'
CONSUMER_SECRET = 'kk9eMhfIMVxoDEoKR63ddWooW87Ya7IgUt5oC31S0TpAXeiMdh'
OAUTH_TOKEN = '917762487608659970-G1v4Nr01JQA9UKqO1HP4g4bPwKT7LAr'
OAUTH_TOKEN_SECRET = 'p1Zp4ophwRbRvR5yET3ppXWWg7fEshIyWwby9vTBxR9CF'

# connect to twitter
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)

# batch size depends on Twitter limit, 100 at this time
batch_size = 100

file = 'img_train_plaintext.txt'


In [55]:
data = pd.read_csv(file, sep='\t', encoding = 'utf8', engine='c', header = 0)
data.head(2)

id                                           imgid  \
0  744014442837454848  http://pbs.twimg.com/media/ClNFMNSWgAI7qq0.jpg   
1  742240402288345088  http://pbs.twimg.com/media/CkzuE33WsAA2cUB.jpg   

  annotations  
0      47,883  
1         229

In [56]:
def get_image(url):
    # downloads images as rgb images
    try:
        image = io.imread(url)
        return image
    except:
        return []
    
def filter_tweets(data):
    rows_to_drop = []
    
    for i, tweet in data.iterrows():
        image = get_image(data.iloc[i, 1])
        if image == []:
            rows_to_drop.append(i)
        else:
            # Save image as id.jpg
            try:
                io.imsave(fname= 'images\\'+ str(data.iloc[i, 0]) + '.jpg', arr=image)
            except:
                io.imsave(fname= 'images\\'+ str(data.iloc[i, 0]) + '.png', arr=image)                
    data = data.drop(data.index[rows_to_drop])
    return data            

In [57]:
print(len(data))
data = filter_tweets(data)
print(len(data))


data.to_csv(path_or_buf= 'test' + file,index = False, sep = '\t', encoding='utf8')


916864


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  


916861


In [58]:
data = pd.read_csv('test'+file, sep='\t', encoding = 'utf8', engine='c')
data.head()

id                                           imgid  \
0  744014442837454848  http://pbs.twimg.com/media/ClNFMNSWgAI7qq0.jpg   
1  747888709375229952  http://pbs.twimg.com/media/CmEIe-_VYAADvg0.jpg   
2  747199027649871872  http://pbs.twimg.com/media/Cl6UJF1XIAADDAJ.jpg   
3  743426787267338240  http://pbs.twimg.com/media/ClEupvHUYAA0lPF.jpg   
4  745081041631612928  http://pbs.twimg.com/media/ClaNsdLWkAAHti4.jpg   

  annotations  
0      47,883  
1   1108,1108  
2        1466  
3        1381  
4        1380

In [16]:
def get_tweets(twapi, data):
    '''
    Fetches content for tweet IDs in a file using bulk request method,
    which vastly reduces number of HTTPS requests compared to above;
    however, it does not warn about IDs that yield no tweet.

    `twapi`: Initialized, authorized API object from Tweepy
    '''    
    tweet_ids = data.id.values.tolist()
    emoji_labels = data.annotations.values.tolist()
    
    all_tweets = []
    i = 0
    # process remainder
    while len(tweet_ids) > 0:
        if len(tweet_ids) < batch_size:
            tweets = twapi.statuses_lookup(id_ = tweet_ids, include_entities=False, trim_user=True)
            tweet_ids = []
        else:
            tweets = twapi.statuses_lookup(id_ = tweet_ids[:batch_size], include_entities=False, trim_user=True)
            tweet_ids = tweet_ids[batch_size:]
        
        for tweet in tweets:
            text = re.sub(r'http\S+', '', tweet.text).strip(' ')
            all_tweets.append(np.array([text, coordinates[0], coordinates[1]]))
            
        i += 1 
        if i == 2:
            break
    return np.array(all_tweets)

In [18]:
all_tweets = get_tweets(api, data)
all_tweets

array([['⚠️ Severe Thunderstorm Warning including Alliance NE, Box Butte Dam NE, Hemingford NE until 7:00 PM MDT',
        '-103.04589153', '42.17673257'],
       ['Woke up @ 8, finally installed Spotify... where did 6h dissappear? When am I? 😞',
        '20.45933301', '44.80418954'],
       ['⚠️ Severe Thunderstorm Warning including Sauk Centre MN, Glenwood MN, Brooten MN until 6:15 PM CDT',
        '-95.15013405', '45.6796762'],
       ['Working solo at 12th and D SW today hope to see you soon 😅',
        '-77.02814218', '38.88430851'],
       ['⚠️ Severe Thunderstorm Warning including Flagler CO until 7:45 PM MDT',
        '-103.00328199', '39.25625525'],
       ['⚠️ Severe Thunderstorm Warning including Kossuth IN, Mckinley IN, Plattsburg IN until 4:00 PM EDT',
        '-86.19914686', '38.6670144']], dtype='<U103')